# Хроматическое число R^4

### 1. Описание алгоритма

Алгоритм:

- строим решетку (grid) и соответствующее ей разбиение Вороного (vor4)
- Ищем все многогранники центрального региона и его диаметр
- составляем список граней центрального многогранника (list_faces)
- перебираю различные подрешетки для каждой ищу приведенный базис, нахожу длину минимального вектора и минимальное расстояние между векторами и их комбинациями:
- Ищем точку s (середина расстояния между точкой (0, 0, 0, 0) и центром ближайшего многогранника
- Нахожу расстояние от точки s до центрального многоранника
- Умножаю это расстояние на 2 и получаю расстояние между многранниками (центральным и ближайшим к нему многогранником из подрешетки)
- составляем списки точек s , расстояния между многогранниками и подрешеток


- grid - изначальная решетка
- central - список вершин центрального многогранника
- coords4 - список координат центров многогранников Вороного
- centr_index - индекс центрального региона (в coords4)
- vor4.vertices - список координат вершин разбиенния Вороного
- vor4.regions - список индексов координат вершин многогранников разбиения Вороного (vor4.vertices)
- vor4.ridge_vertices - координаты вершин 3-мерных граней разбиенния Вороного через их индексы в vor4.vertices
- vor4.ridge_points - список пар центров многогранников Вороного между которыми есть ребро 2х многогранников через индексы в coords4 (по сути - соседние многогранники)

- edge_central_coords - список списков вершин каждой 3-мерной грани центрального многогранника
- faces_3d - список индексов (в vor4.vertices)  координат каждой 3-мерной грани 
- faces_2d - список индексов (в vor4.vertices)  координат каждой 2-мерной грани 
- edges - список индексов (в vor4.vertices)  координат каждого ребра
- list_faces - список 3х мерных граней центрального многогранника с разбивкой каждой 3х мерной грани на 2-мерные


- neighbor_coordinates - список координат центров многогранников, граничащих с центральным (по сути - вектора нормали к 3-мерным граням центрального многогранника) - координаты
 - list_neigh_points_ind - список координат центров многогранников, граничащих с центральным (по сути - вектора нормали к 3-мерным граням центрального многогранника) - индексы
- list_ridge_edge - список граней, в том же порядке, что и соседние точки (list_neigh_points) или что и соответствующие вектора нормали к данным граням
- v_norm - список нормированных векторов нормали (в том же порядке, что и изначальные точки(вектора) нормали)
- vertex_to_faces = собой список списков 3D-граней для каждой вершины (где индексы граней берутся из edge_central_coords)
- point_face_norm - точки, принадлежащие грани и лежащие на векторе нормали (середина вектора, соединяющего центры)
- polyhedrons - данные класса центрального многогранника: грани(2d и 3d), ребра, вектора нормали, центры 

### 2. Подключение библиотек

In [ ]:
import numpy as np
from scipy.spatial import Voronoi
from scipy.spatial import distance
from sympy import symbols, Eq, solve
import math
import itertools
import copy
from scipy.spatial import Delaunay
import pandas as pd
import sys
from numpy import linalg as la
from itertools import *
from copy import deepcopy
from collections import defaultdict

from itertools import combinations
from math import prod
from sympy import factorint

import networkx as nx
from scipy.spatial import ConvexHull

from scipy.spatial import distance
from fpylll import IntegerMatrix, LLL

import time

/Users/konstantinglushkov/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### 3. Построение многогранника Вороного

In [81]:

grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)
'''
grid = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
], dtype = float)

grid = np.array([
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1],
    [0.5, 0.5, 0.5, 0.5]
], dtype = float)
'''

grid = np.array([
    [1, 1, 0, 0],
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1]
], dtype = float)

coords4 = [] # координаты центров

for var in itertools.product(range(-3, 3), repeat=4): # меньше 3 нельзя - тк при поиске центр региона отбрасываем незаконченные регионы
    coords4.append((grid.T).dot(var).tolist())
    
# строим диаграмму Вороного
vor4 = Voronoi(coords4)

может быть убрать, тк regions_finite не используется дальше 

In [82]:
# список конечных областей Вороного (-1 в координатах означает, что область бесконечна)

count = 0
vor4.regions_finite = []
for region in vor4.regions:
    if -1 not in region:
        list_coord_reg = []
        for vert in region:
            list_coord_reg.append(vor4.vertices[vert])
        vor4.regions_finite.append(list_coord_reg)
        count += 1

vor4.regions_finite.pop(0)  # удаляем первый пустой регион 
      

[]

### 4. Построение центрального региона

может быть убрать тк далее не используем

In [6]:
# максимальный размер региона (по числу вершин)

len_r_max = 0 # размер максимального региона
count_r = 0 # количество таких регионов

for region in vor4.regions:
    len_r = len(region)
    
    if len_r >= len_r_max:
        len_r_max = len_r
        count_r += 1 # считаем количество таких регионов
        
print('количество вершин =', len_r_max, 'количество таких регионов =', count_r)

количество вершин = 24 количество таких регионов = 257


for i in vor4.ridge_vertices:
    if len(i) < 6 and -1 not in i:
        display(i)

In [83]:
# находим суммарные расстояния от вершин многогранников до (0, 0, 0, 0) (sum_dist), затем находим минимальное 
# значение. Таким образом находим ближайший центральный многогранник central

sum_dist_min = 1000

for region in vor4.regions:
    
    len_r = len(region)

    if -1 in region or len(region) == 0: continue # регион неограничен
        
    length = 0
    
    # для каждого региона перебираем все его вершины и для каждой ищем расстояние до [0.0, 0.0, 0.0, 0.0]
    for j in range(len_r):
        length += distance.euclidean(vor4.vertices[region[j]], [0.0, 0.0, 0.0, 0.0])

    if length < sum_dist_min:
        sum_dist_min = length
        v_min = region

# ищу индекс минимального региона
v_min = next((i for i, arr in enumerate(vor4.regions) if np.array_equal(arr, v_min)), None)

print ('суммарное расстояние =', sum_dist_min, 'индекс центрального региона =', v_min)

central = vor4.vertices[vor4.regions[v_min]]
vor4.central_index = vor4.regions[v_min] # индексы координат (в coord4) центрального региона
vor4.central = vor4.vertices[vor4.regions[v_min]]


суммарное расстояние = 24.0 индекс центрального региона = 1294


 наверное эту колонку надо удалить - не используется дальше

In [84]:
# нахожу длину ребра центрального политопа

d_edge_min = 100
count = 0
for i in range(0, len(central)-1):
    for j in range(i + 1, len(central)):
        d_edge = round(distance.euclidean(central[i], central[j]), 10)
        if d_edge_min >= d_edge:
            d_edge_min = d_edge
            count += 1
            
d_edge_min, count
                           

(1.0, 96)

In [85]:
# нашли индекс точки(центра) центрального многогранника

centr_index = coords4.index([0.0, 0.0, 0.0, 0.0])
coords4[centr_index]

[0.0, 0.0, 0.0, 0.0]

In [86]:
'''
центральный регион по индексу (point_region - список индексов индексов регионов в том же порядке, что и 
исходные точки (coord4). Те, зная индекс точки в coord4, можно найти индекс региона в regions по списку в
point_region)
'''

vor4.regions[vor4.point_region[centr_index]]

[1035,
 1270,
 1273,
 1279,
 1622,
 1627,
 1779,
 1782,
 1783,
 1784,
 1786,
 1796,
 1806,
 2448,
 2456,
 2634,
 2635,
 2636,
 2735,
 2736,
 2739,
 2747,
 2749,
 2794]

In [87]:
# находим 3d грани

vor4.central_3d = [] # список 3d граней центрального региона
faces_3d = []

for ridge in vor4.ridge_vertices:
    
    if np.all(np.isin(ridge, vor4.central_index)):
        vor4.central_3d.append(vor4.vertices[ridge])
        faces_3d.append(ridge)

# в данный список могли попасть не только 3х мерные грани
# если какая то грань полностью входит в другую, то это не 3х мерная грань

to_remove = [] # список не 3d граней (список граней к удалению из зх мерных граней)

for face_1 in faces_3d:

    for face_2 in faces_3d:
        # если face_1 входит в face_2 и длина face_1 меньше  face_2, то face_1 не 3х мерная грань
        if np.all(np.isin(face_1, face_2)) and len(face_1) < len(face_2):
            to_remove.append(face_1)
            break
                
#  удаляем лишние грани
for face in to_remove:    
    faces_3d.remove(face)

    
faces_3d

[[1035, 1270, 1273, 1779, 1783, 1786],
 [1279, 1782, 1786, 1796, 1806, 2448],
 [1622, 1627, 1779, 1782, 1784, 2456],
 [1779, 1782, 1783, 1784, 1786, 1806],
 [1783, 1784, 1806, 2634, 2635, 2636],
 [1270, 1783, 1786, 1796, 1806, 2634],
 [1035, 1627, 1779, 1783, 1784, 2635],
 [1782, 1784, 1806, 2448, 2456, 2636],
 [1273, 1279, 1622, 1779, 1782, 1786],
 [1035, 1270, 1273, 2735, 2739, 2749],
 [1270, 1796, 2634, 2735, 2736, 2739],
 [1279, 1796, 2448, 2735, 2736, 2747],
 [1273, 1279, 1622, 2735, 2747, 2749],
 [1270, 1273, 1279, 1786, 1796, 2735],
 [1796, 1806, 2448, 2634, 2636, 2736],
 [1279, 1622, 1782, 2448, 2456, 2747],
 [1035, 1273, 1622, 1627, 1779, 2749],
 [1035, 1270, 1783, 2634, 2635, 2739],
 [1627, 1784, 2456, 2635, 2636, 2794],
 [1035, 1627, 2635, 2739, 2749, 2794],
 [1622, 1627, 2456, 2747, 2749, 2794],
 [2448, 2456, 2636, 2736, 2747, 2794],
 [2634, 2635, 2636, 2736, 2739, 2794],
 [2735, 2736, 2739, 2747, 2749, 2794]]

In [88]:
# переводим в координаты индексы 3d грани

edge_central_coords = []  # Координаты вершин 3D-граней

for face in faces_3d:
    list_c = []
    for vert in face:
        list_c.append(vor4.vertices[vert])
    #vor4.vertices[faces_3d]
    edge_central_coords.append(list_c)
edge_central_coords

[[array([-0.5,  0.5, -0.5,  0.5]),
  array([-0.5, -0.5, -0.5,  0.5]),
  array([ 0.,  0., -1.,  0.]),
  array([-0.5,  0.5, -0.5, -0.5]),
  array([-1.,  0.,  0.,  0.]),
  array([-0.5, -0.5, -0.5, -0.5])],
 [array([ 0.5, -0.5, -0.5, -0.5]),
  array([ 0.,  0.,  0., -1.]),
  array([-0.5, -0.5, -0.5, -0.5]),
  array([ 0., -1.,  0.,  0.]),
  array([-0.5, -0.5,  0.5, -0.5]),
  array([ 0.5, -0.5,  0.5, -0.5])],
 [array([ 0.5,  0.5, -0.5, -0.5]),
  array([0., 1., 0., 0.]),
  array([-0.5,  0.5, -0.5, -0.5]),
  array([ 0.,  0.,  0., -1.]),
  array([-0.5,  0.5,  0.5, -0.5]),
  array([ 0.5,  0.5,  0.5, -0.5])],
 [array([-0.5,  0.5, -0.5, -0.5]),
  array([ 0.,  0.,  0., -1.]),
  array([-1.,  0.,  0.,  0.]),
  array([-0.5,  0.5,  0.5, -0.5]),
  array([-0.5, -0.5, -0.5, -0.5]),
  array([-0.5, -0.5,  0.5, -0.5])],
 [array([-1.,  0.,  0.,  0.]),
  array([-0.5,  0.5,  0.5, -0.5]),
  array([-0.5, -0.5,  0.5, -0.5]),
  array([-0.5, -0.5,  0.5,  0.5]),
  array([-0.5,  0.5,  0.5,  0.5]),
  array([0., 0., 1., 

In [89]:
# формируем список 2d граней (если у 2х 3d граней есть общие вршины и из более 2, то это 2d грань)

faces_2d  = [[] for _ in range(len(faces_3d))] # список 2d граней с разбивкой по 3d граням

# берем все комбиации 3d граней по 2 и ищем общие вершины
for comb in combinations(range(len(faces_3d)), 2):
    common_coords = list(set(faces_3d[comb[0]]) & set(faces_3d[comb[1]]))
    
    if len(common_coords) > 2:
        faces_2d[comb[0]].append(common_coords)
        faces_2d[comb[1]].append(common_coords)
        #print(common_coords, y[comb[0]], y[comb[1]])
        
faces_2d

[[[1786, 1779, 1783],
  [1786, 1270, 1783],
  [1783, 1035, 1779],
  [1273, 1786, 1779],
  [1273, 1035, 1270],
  [1273, 1786, 1270],
  [1273, 1035, 1779],
  [1035, 1270, 1783]],
 [[1782, 1786, 1806],
  [1786, 1796, 1806],
  [2448, 1782, 1806],
  [1786, 1782, 1279],
  [2448, 1796, 1279],
  [1786, 1796, 1279],
  [2448, 1796, 1806],
  [2448, 1782, 1279]],
 [[1784, 1779, 1782],
  [1784, 1627, 1779],
  [2456, 1784, 1782],
  [1622, 1779, 1782],
  [2456, 1622, 1782],
  [1627, 1779, 1622],
  [2456, 1784, 1627],
  [2456, 1627, 1622]],
 [[1786, 1779, 1783],
  [1782, 1786, 1806],
  [1784, 1779, 1782],
  [1784, 1806, 1783],
  [1786, 1806, 1783],
  [1784, 1779, 1783],
  [1782, 1784, 1806],
  [1786, 1779, 1782]],
 [[1784, 1806, 1783],
  [2634, 1806, 1783],
  [1784, 2635, 1783],
  [1784, 2636, 1806],
  [2634, 2636, 1806],
  [2634, 2635, 1783],
  [1784, 2635, 2636],
  [2634, 2635, 2636]],
 [[1786, 1270, 1783],
  [1786, 1796, 1806],
  [1786, 1806, 1783],
  [2634, 1806, 1783],
  [2634, 1796, 1270],
  [17

In [90]:
# поиск ребер для 2х мерной грани

def find_edges(vert_face_2d):
   
    list_ind = [] # список вершин в индексах
    list_edges_face_2d_ind = [] # список ребер грани в индексах
    list_edges_face_2d_coord = [] # список ребер грани в координатах
    
    for vert in vert_face_2d:
        #print('v', vert)
        ind = np.argwhere((vor4.vertices == vert).all(axis=1))[0][0]
        #print('ind', ind)
        list_ind.append(ind)
        
    for edge in list_edges:
        if edge[0] in list_ind and edge[1] in list_ind:
            list_edges_face_2d_ind.append(edge)
            
    
    return vor4.vertices[list_edges_face_2d_ind]
            

In [91]:
# ищем ребра (индексы)

edges  = [[] for _ in range(len(faces_2d))]

for index in range(len(faces_2d)):
    
    for comb in combinations(range(len(faces_2d[index])), 2):
        common_coords = list(set(faces_2d[index][comb[0]]) & set(faces_2d[index][comb[1]]))
        
        if len(common_coords) == 2:
            common_coords.sort()
            edges[index].append(common_coords)
            
        if len(common_coords) > 2:
            print('Error')

edges       

[[[1783, 1786],
  [1779, 1783],
  [1779, 1786],
  [1270, 1786],
  [1270, 1783],
  [1035, 1779],
  [1035, 1783],
  [1273, 1786],
  [1273, 1779],
  [1270, 1273],
  [1035, 1273],
  [1035, 1270]],
 [[1786, 1806],
  [1782, 1806],
  [1782, 1786],
  [1786, 1796],
  [1796, 1806],
  [1806, 2448],
  [1782, 2448],
  [1279, 1786],
  [1279, 1782],
  [1279, 1796],
  [1796, 2448],
  [1279, 2448]],
 [[1779, 1784],
  [1782, 1784],
  [1779, 1782],
  [1627, 1779],
  [1627, 1784],
  [1782, 2456],
  [1784, 2456],
  [1622, 1782],
  [1622, 1779],
  [1622, 2456],
  [1622, 1627],
  [1627, 2456]],
 [[1783, 1786],
  [1779, 1783],
  [1779, 1786],
  [1786, 1806],
  [1782, 1806],
  [1782, 1786],
  [1779, 1784],
  [1782, 1784],
  [1779, 1782],
  [1783, 1806],
  [1783, 1784],
  [1784, 1806]],
 [[1783, 1806],
  [1783, 1784],
  [1784, 1806],
  [1806, 2634],
  [1783, 2634],
  [1783, 2635],
  [1784, 2635],
  [1806, 2636],
  [1784, 2636],
  [2634, 2636],
  [2634, 2635],
  [2635, 2636]],
 [[1783, 1786],
  [1270, 1786],
  [

In [92]:
# считаем количество ребер

list_edges = []

for face in edges:
    for edge in face:
        if edge not in list_edges:
            list_edges.append(edge)
            
len(list_edges)

96

In [93]:
# составляем список центров многогранников, граничащих с центральным
# индексы координат в coords4

list_neigh_points = [] # список центров многогранников, граничащих с центральным
vor4.list_neigh_points = []
list_neigh_points_ind = [] # индексы из coords4

for pair in vor4.ridge_points:

    if centr_index not in pair: continue

    if pair[0] == centr_index:
        
        # если в списке 4 многогранника есть вершины хоть одой 3 грани центрального многогранника
        # то этот многогранник соседний с центральным
        if any(all(item in vor4.regions[vor4.point_region[pair[1]]] for item in small) for small in faces_3d):
            list_neigh_points.append(coords4[pair[1]])
            vor4.list_neigh_points.append(coords4[pair[1]])
            list_neigh_points_ind.append(pair[1])
           
    else:
        if any(all(item in vor4.regions[vor4.point_region[pair[0]]] for item in small) for small in faces_3d):
            list_neigh_points.append(coords4[pair[0]])
            vor4.list_neigh_points.append(coords4[pair[0]])
            list_neigh_points_ind.append(pair[0])

In [94]:
# переводим индекс соседних центров в координаты

neighbor_coordinates = []  # Координаты соседних точек

for i in list_neigh_points_ind:
    neighbor_coordinates.append(coords4[i])
    #print(coords4[i])
    
neighbor_coordinates

[[-1.0, 0.0, -1.0, 0.0],
 [0.0, -1.0, 0.0, -1.0],
 [0.0, 1.0, 0.0, -1.0],
 [-1.0, 0.0, 0.0, -1.0],
 [-1.0, 0.0, 1.0, 0.0],
 [-1.0, -1.0, 0.0, 0.0],
 [-1.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, -1.0],
 [0.0, 0.0, -1.0, -1.0],
 [0.0, 0.0, -1.0, 1.0],
 [0.0, -1.0, 0.0, 1.0],
 [1.0, -1.0, 0.0, 0.0],
 [1.0, 0.0, -1.0, 0.0],
 [0.0, -1.0, -1.0, 0.0],
 [0.0, -1.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, -1.0],
 [0.0, 1.0, -1.0, 0.0],
 [-1.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 1.0],
 [1.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 1.0],
 [1.0, 0.0, 0.0, 1.0]]

In [95]:
# ищем для каждого центра ближайшую трехмерную грань
# берем каждую точку из списка соседних центральных точек и ищем расстояние от нее до каждой 3х мерной грани
# суммируем и берем минимальное ссумарное расстояние. это и будет 3х мерная грань между точкой (0, 0, 0, 0)
# точкой из списка
# и вектор, соединяющий центры будет вектором нормали к данной 3х мерной грани


#dict_point_edge = {} # словарь: ключ - соседняя точка, значение - 3х мерная грань между соседней точкой и (0,0,0,0)
#index_point_edge = 0 # индекс грани в списке граней центрального многогранника (edge_central_coords[)
list_ridge_edge = [] # список граней, в том же порядке, что и соседние точки (list_neigh_points)


# перебираем все соседние центры

for point in list_neigh_points:
    
    min_dist_point_edge = 1000

    # для каждой 3х мерной грани   
    for edge in edge_central_coords:
        
        dist_point_edge = 0
        
        # для каждой вершины их 3х мерной грани считаем растояние до соседнего центра и суммируем
        for coord in edge:
            
            dist_point_edge += distance.euclidean(coord, point)
            
        # если суммарное растояние меньше минимального, записываем новое суммарное растояние и индекс грани    
        if min_dist_point_edge > dist_point_edge:
            min_dist_point_edge = dist_point_edge
            point_edge = edge

    v_min = next((i for i, arr in enumerate(vor4.regions) if np.array_equal(arr, v_min)), None)


    #dict_point_edge[tuple(point)] = point_edge
    list_ridge_edge.append(edge)
    


In [96]:
# строим список 2d граней 3d граней
list_faces = []
list_vert_3d = []
for face3d in faces_2d:
    list_vert_3d = []
    
    for face2d in face3d:
        list_verts = []
        for vert in face2d:
            list_verts.append(vor4.vertices[vert])
        list_vert_3d.append(list_verts) 
    list_faces.append(list_vert_3d)
list_faces

[[[array([-0.5, -0.5, -0.5, -0.5]),
   array([-0.5,  0.5, -0.5, -0.5]),
   array([-1.,  0.,  0.,  0.])],
  [array([-0.5, -0.5, -0.5, -0.5]),
   array([-0.5, -0.5, -0.5,  0.5]),
   array([-1.,  0.,  0.,  0.])],
  [array([-1.,  0.,  0.,  0.]),
   array([-0.5,  0.5, -0.5,  0.5]),
   array([-0.5,  0.5, -0.5, -0.5])],
  [array([ 0.,  0., -1.,  0.]),
   array([-0.5, -0.5, -0.5, -0.5]),
   array([-0.5,  0.5, -0.5, -0.5])],
  [array([ 0.,  0., -1.,  0.]),
   array([-0.5,  0.5, -0.5,  0.5]),
   array([-0.5, -0.5, -0.5,  0.5])],
  [array([ 0.,  0., -1.,  0.]),
   array([-0.5, -0.5, -0.5, -0.5]),
   array([-0.5, -0.5, -0.5,  0.5])],
  [array([ 0.,  0., -1.,  0.]),
   array([-0.5,  0.5, -0.5,  0.5]),
   array([-0.5,  0.5, -0.5, -0.5])],
  [array([-0.5,  0.5, -0.5,  0.5]),
   array([-0.5, -0.5, -0.5,  0.5]),
   array([-1.,  0.,  0.,  0.])]],
 [[array([ 0.,  0.,  0., -1.]),
   array([-0.5, -0.5, -0.5, -0.5]),
   array([-0.5, -0.5,  0.5, -0.5])],
  [array([-0.5, -0.5, -0.5, -0.5]),
   array([ 0., -1.

In [112]:
# диаметр многогранника (ищем максимальное расстояние между вершинами) - 2

max_len = round(2 * distance.euclidean(np.array([0, 0, 0, 0]), central[1]), 10)
max_len

2.0

In [98]:
# нормируем вектора нормали
# ищем точки, принадлежащие граням и находящиеся на векторах нормали

v_norm = [] # здесь будем хранить нормированные вектора нормали
point_face_norm = [] #точки, принадлежащие граням и находящиеся на векторах нормали

for vec in neighbor_coordinates:
    v_norm.append(np.array(vec / np.linalg.norm(vec)))
    point_face_norm = np.array(vec) / 2
        

In [99]:
vertex_to_faces = [] # собой список списков 3D-граней для каждой вершины 
                        # (где индексы граней берутся из edge_central_coords)
'''    
берем каждую верщину из central и перебираем все 3х мерные грани - смотрим есть ли данная вершина в этой 
3х мерной грани если да, то тдобавляем ее в список
'''
for vertex in central:
    
    tamp_list = [] # список граней для текущей вершины
    
    for face in range(len(edge_central_coords)):

        if np.any(np.all(edge_central_coords[face] == vertex, axis=1)):
            tamp_list.append(face)
            
    vertex_to_faces.append(tamp_list)


### 5. Триангуляция Делоне

In [100]:
# строим триангуляцию

delaunay = Delaunay(central)

In [101]:
# Получаем все вершины симплексов
simplices_vertices = delaunay.simplices

# Уникальные вершины
unique_vertices = np.unique(simplices_vertices)

# Проверяем, что все точки из central используются
if len(unique_vertices) == len(central):
    print('Все точки используются в триангуляции.')
else:
    print('Не все точки используются в триангуляции.')

Все точки используются в триангуляции.


In [102]:
# Выпуклая оболочка исходных точек
original_hull = ConvexHull(central)

# Все точки триангуляции
triangulated_points = central[delaunay.simplices.flatten()]

# Выпуклая оболочка точек триангуляции
triangulated_hull = ConvexHull(triangulated_points)

# Сравнение объемов выпуклых оболочек
is_convex_hull_correct = np.isclose(original_hull.volume, triangulated_hull.volume)
print('Выпуклость триангуляции сохранена:', is_convex_hull_correct)

Выпуклость триангуляции сохранена: True


### 6. Создание классов граней и ребер центрального многогранника

In [103]:
class Edge2D:
    def __init__(self, vertex1, vertex2, face_center):
        '''
        Инициализация ребра.
        :vertex1: Координаты первой вершины ребра.
        :vertex2: Координаты второй вершины ребра.
        :face_center: Координаты центра грани.
        '''
        self.vertex1 = np.array(vertex1)
        self.vertex2 = np.array(vertex2)
        self.face_center = np.array(face_center)

        # Вычисляем середину ребра
        self.center = (self.vertex1 + self.vertex2) / 2

        # Вычисляем нормаль к ребру
        #edge_vector = self.vertex2 - self.vertex1
        self.normal = self.center - self.face_center  # Перпендикулярный вектор
        self.normal = self.normal / np.linalg.norm(self.normal)  # Нормализация
        
        self.bias = self.normal @ vertex1

        # Проверяем направление нормали (она должна быть направлена от центра грани)
        vector_to_center = self.face_center - self.center
        if np.dot(self.normal, vector_to_center) > 0:
            self.normal = -self.normal  # Меняем направление нормали

    def __repr__(self):
        return f"Edge2D(vertex1={self.vertex1}, vertex2={self.vertex2}, normal={self.normal}, \
                midpoint={self.center})"

In [104]:
class Face2D:
    def __init__(self, vertices, polyhedron_center):
        '''
        Инициализация 2D грани многогранника.
        '''
        self.vertices = vertices  # Координаты вершин грани
        self.parent_center = polyhedron_center # центр 3х мерной грани
        self.center = self._calculate_center()  # Центр грани
        self.normal = self._calculate_normal()  # Нормаль к грани
        self.edges = self._calculate_edges()  # ребра в 2х мерном многограннике
        self.bias = self.normal @ vertices[0]

    def _calculate_center(self):
        '''
        Вычисляет центр грани как среднее арифметическое координат вершин.
        '''
        return tuple(np.mean(self.vertices, axis=0))

    def _calculate_normal(self):

        # Вычисляем нормаль к грани как вектор из центра 3х мерного многогранника в центр 2х мерной грани
        normal_3d = np.array(self.center) - np.array(self.parent_center)

        # Нормализуем нормаль
        normal_3d = normal_3d / np.linalg.norm(normal_3d)

        return normal_3d

    def _calculate_edges(self):
        
        # составляем список ребер (расстояние между вершинами, соединенными ребрами = 1)
        list_edges = []
        
       
        found_edges = find_edges(self.vertices)
        
        for vertex1, vertex2 in found_edges:
            list_edges.append(Edge2D(vertex1, vertex2, self.center))
            
        return list_edges

    
    def __repr__(self):
        '''
        Возвращает строковое представление грани.
        '''
        return (f"Face2D(vertices={self.vertices}), center={self.center}, "
                f"normal={self.normal}")

In [ ]:

class Face3D:
    def __init__(self, vertices, faces_list, normal):
        '''
        Инициализация многогранника в 4-мерном пространстве.
        :vertices: Список координат вершин многогранника .
        :normal: Вектор нормали к многограннику.
        '''
        self.vertices = vertices  # Координаты вершин
        self.normal = normal  # Нормаль к многограннику
        self.faces_list = faces_list  # 2d Грани 3d многогранника
        self.center = self._calculate_center()  # Центр многогранника
        self.faces = self._find_faces() # грани многогранника (как экземпляр класса)
        self.bias = normal @ vertices[0] # смещение

    def _calculate_center(self):
        '''
        Вычисляет центр многогранника как среднее арифметическое координат вершин.
        :return: Координаты центра.
        '''
        return tuple(np.mean(self.vertices, axis=0))

    
    def _find_faces(self):
        '''
        Находит грани многогранника и создаёт объекты Face2D.
        :return: Список объектов Face2D.
        '''
        
        faces = []

        for face_vertices in self.faces_list:
            face = Face2D(face_vertices, self.center)
            faces.append(face)
            
        #for index in range(len(self.faces_list)):
            #face = Face2D(self.faces_list, index, self.center)
            #faces.append(face)

        return faces
    

    def __repr__(self):
        '''
        Возвращает строковое представление многогранника.
        '''
        return (f"Face3D(vertices={self.vertices}, normal={self.normal}, "
                f"center={self.center}, faces={self.faces})")


In [106]:
# расчитываем все грани центрального многогранника

polyhedrons = []  # Список для хранения всех объектов

for face_index in range(len(edge_central_coords)):
    polyhedron = Face3D(edge_central_coords[face_index], list_faces[face_index], v_norm[face_index])
    polyhedrons.append(polyhedron)
    

### 7. Расстояние от точки s до центрального многогранника

In [107]:
# через минимальное расстояние до вершины

def dist_to_s(polyhedrons, s):
    
    min_dist_to_pol = float('inf')
    index_proj = -1
    min_dist_vert_to_s = float('inf')
    min_vert = -1

    # находим ближайшую вершину к точке s
    
    for index in range(len(central)):

        dist_vert_to_s = distance.euclidean(s, central[index])

        if dist_vert_to_s < min_dist_vert_to_s:
            min_dist_vert_to_s = dist_vert_to_s
            min_vert = index
        

    # находим расстояние и проекцию на центральный многогранник
    
    for i in vertex_to_faces[min_vert]: # рассматриваем только грани,которым принадлежит ближайшая в точке s вершина
        # coord0 = s - (d0 + polyhedrons[i].bias)* polyhedrons[i].normal
        d0 = polyhedrons[i].normal @ (s - polyhedrons[i].center)
        coord0 = s - d0 * polyhedrons[i].normal
        simplex = delaunay.find_simplex(coord0)
        
        if simplex != -1: # если проекция принадлежит центральному многораннику
            dist = abs(d0)
            coords_to_central = coord0

            if min_dist_to_pol > dist:
                min_dist_to_pol = dist
                coords_proj = coords_to_central
                index_proj = i

        else:
            for face2d in polyhedrons[i].faces: #cycle

                d1 = face2d.normal @ (coord0 - face2d.center)
                # coord1 = coord0 - (d1 + face2d.bias)* polyhedrons[i].normal
                coord1 = coord0 - d1 * face2d.normal
                simplex = delaunay.find_simplex(coord1)

                if simplex != -1: # если проекция принадлежит центральному многораннику
                    dist = distance.euclidean(s, coord1)#, dtype = 'float')
                    coords_to_central = coord1

                    if min_dist_to_pol > dist:
                        min_dist_to_pol = dist
                        coords_proj = coords_to_central
                        index_proj = i

                else:

                    for edge in face2d.edges:
                        d2 = edge.normal @ (coord1 - edge.center)
                        #coord2 = coord1 - (d2 + edge.bias) * edge.normal
                        coord2 = coord1 - d2 * edge.normal
                        simplex = delaunay.find_simplex(coord2)

                        if simplex != -1: # если проекция принадлежит центральному многораннику
                            dist = distance.euclidean(s, coord2)#, dtype = 'float')
                            coords_to_central = coord2

                            if min_dist_to_pol > dist:
                                min_dist_to_pol = dist
                                coords_proj = coords_to_central
                                index_proj = i

                        else:
                            d3 = distance.euclidean(s, edge.vertex1)
                            d4 = distance.euclidean(s, edge.vertex2)

                            if d3 < d4:
                                dist = d3
                                coords_to_central = edge.vertex1
                            else:
                                dist = d4
                                coords_to_central = edge.vertex2

                            if min_dist_to_pol > dist:
            
                                min_dist_to_pol = dist
                                coords_proj = coords_to_central
                                index_proj = i
        
        
        # если расстояние до какой-либо грани < 1, то дальше не считаем
        if  min_dist_to_pol < 1:

            return min_dist_to_pol, coords_proj, index_proj

        
    return min_dist_to_pol * 2 / max_len, coords_proj, index_proj

In [108]:
min_d = 1

### 8. LLL-базис (функция)

In [ ]:
def gram_schmidt(basis):
    n = len(basis)
    b_star = [np.array(basis[i], dtype=float) for i in range(n)]
    mu = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i):
            mu[i,j] = np.dot(basis[i], b_star[j]) / np.dot(b_star[j], b_star[j])
            b_star[i] -= mu[i,j] * b_star[j]
        mu[i,i] = np.dot(b_star[i], b_star[i])  
    
    return b_star, mu


def size_reduction(basis, b_star, mu, k, l):
    '''Выполняет Size-Reduction шаг.'''
    q = round(mu[k, l])
    basis[k] -= q * basis[l].copy()
    mu[k, l] -= q
    for j in range(l):
        mu[k, j] -= q * mu[l, j]


def interchange(basis, mu, b_star, beta, k):
    '''Выполняет Interchange шаг.'''
    # Меняем местами векторы
    basis[k], basis[k - 1] = basis[k - 1].copy(), basis[k].copy()
    
    # Обновляем mu
    for j in range(k - 1):
        mu[k, j], mu[k - 1, j] = mu[k - 1, j], mu[k, j]
    
    mu_k_km1 = mu[k, k - 1]
    beta_k = beta[k] - mu_k_km1 ** 2 * beta[k - 1]
    beta_km1 = beta[k - 1]
    
    
    # Обновляем mu
    mu[k, k - 1] = mu_k_km1 * beta_km1 / beta_k
    b = b_star[k-1]
    b_star[k-1] = b_star[k] + mu_k_km1 * b
    b_star[k] = (beta[k]/beta_k) * b - mu_k_km1 * b_star[k]
    
    # Обновляем beta
    beta[k] = beta[k - 1] * beta[k] / beta_k
    beta[k - 1] = beta_k
    
    
    for i in range(k + 1, len(basis)):
        temp = mu[i, k]
        mu[i, k] = mu[i, k - 1] - mu_k_km1 * temp
        mu[i, k - 1] = temp + mu[k, k-1] * mu[i, k]


def LLL(basis, delta=0.75):
    n = len(basis)
    basis = [np.array(vec, dtype=float) for vec in basis]
    b_star, mu = gram_schmidt(basis)
    beta = [np.dot(b_star[i], b_star[i]) for i in range(n)]
    
    k = 1
    while k < n:
        # Size Reduction
        for l in range(k-1, -1, -1):
            if abs(mu[k,l]) > 0.5:
                q = round(mu[k,l])
                basis[k] -= q * basis[l]
                # Обновляем mu
                mu[k,l] -= q
                for j in range(l):
                    mu[k,j] -= q * mu[l,j]
                # Пересчитываем Грама-Шмидта после изменения базиса
                b_star, mu = gram_schmidt(basis)
                beta = [np.dot(b_star[i], b_star[i]) for i in range(n)]
        
        # Проверка условия Ловаса
        if beta[k] < (delta - mu[k,k-1]**2) * beta[k-1]:
            # Interchange
            basis[k], basis[k-1] = basis[k-1].copy(), basis[k].copy()
            # Пересчитываем всё после обмена
            b_star, mu = gram_schmidt(basis)
            beta = [np.dot(b_star[i], b_star[i]) for i in range(n)]
            k = max(1, k-1)
        else:
            k += 1
    
    return np.array(basis)

In [ ]:
# ищет точку s для подрешетки

def s_point(sub_grid):
    digits = np.array(list(product(range(-2, 2), repeat=4)))  # Все комбинации
    coords4_1 = np.dot(digits, sub_grid)  # Все точки подрешётки
    
    # Удаляем нулевую точку
    coords4_1 = coords4_1[~np.all(coords4_1 == 0, axis=1)]
    
    # Минимальное расстояние между точками
    if len(coords4_1) >= 2:
        dist_matrix = distance.pdist(coords4_1)
        min_dist = np.min(dist_matrix)
        if min_dist <= 2:
            return np.array([0, 0, 0, 0])
    
    # Поиск общей точки с coords4
    common_coord = np.array([x for x in coords4_1 if x.tolist() in coords4])
    if len(common_coord) == 0:
        return np.array([0, 0, 0, 0])
    
    # Ближайшая к центру точка
    dist_to_center = np.linalg.norm(common_coord, axis=1)
    closest_idx = np.argmin(dist_to_center)
    return common_coord[closest_idx] * 0.5

### 9. Поиск подрешеток

In [37]:
# генерация комбинаций

def generate_factor_combinations(factors):
    if not factors:
        return [[]]
    
    first, *rest = factors
    rest_combinations = generate_factor_combinations(rest)
    
    result = []
    for comb in rest_combinations:
        result.append([first] + comb)
        result.append([first * (comb[0] if comb else 1)] + (comb[1:] if len(comb) > 1 else []))
    
    return result

In [38]:
# находит все возможные разложения числа на простые множители

def compute_factorizations(n):
    
    # находим простые множители числа
    
    factors = [] # список простых множителей
    
    for prime, exp in factorint(n).items():
        factors.extend([prime] * exp)
    
    combinations = []
    combinations = generate_factor_combinations(factors) # генерируем список всех возможных комбинаций
    
    unique = set(tuple(sorted(comb)) for comb in combinations) # убираем дубликаты и сортируем
    unique_combinations = [list(comb) for comb in sorted(unique)]
    
    unique_combinations_4 = []

    for comb in unique_combinations:

        if len(comb) <= 4:
            
            unique_combinations_4.append(comb)

    
    return unique_combinations_4

    


In [39]:
def pad_lists_with_ones(list_of_lists):
    # Проходим по каждому списку в основном списке
    for i in range(len(list_of_lists)):
        # Если длина списка меньше 4
        if len(list_of_lists[i]) < 4:
            # Добавляем в начало столько единиц, сколько не хватает до 4
            list_of_lists[i] = [1] * (4 - len(list_of_lists[i])) + list_of_lists[i]
    return list_of_lists


In [ ]:
# поиск приведенного базиса через библиотеку fpylll

list_mat = []
list_lll = []
list_s1 = []
list_dist = []

norm_length = 1 / max_len

for det_mat in range(45, 46):

    print(det_mat)

    list_all_factorizations = compute_factorizations(det_mat)
    list_diag_el = pad_lists_with_ones(list_all_factorizations)

    for diag_el in list_diag_el:
        mat = np.array([[diag_el[0], 0, 0, 0], 
                        [0, diag_el[1], 0, 0], 
                        [0, 0, diag_el[2], 0], 
                        [0, 0, 0, diag_el[3]]], int)
        max_num_col3 = diag_el[3]
        max_num_col2 = diag_el[2]
        max_num_col1 = diag_el[1]
        #max_num_col0 = diag_el[0]
        print(max_num_col1, max_num_col2, max_num_col3)

        for i1 in range(max_num_col3):
            mat[2][3] = i1
            print(i1)
            for i2 in range(max_num_col3):
                mat[1][3] = i2
                for i3 in range(max_num_col2):
                    mat[1][2] = i3
                    for i4 in range(max_num_col3):
                        mat[0][3] = i4
                        for i5 in range(max_num_col2):
                            mat[0][2] = i5
                            for i6 in range(max_num_col1):
                                mat[0][1] = i6
                                
                                sub_grid = np.dot(mat, grid)
                                sub_grid_int = (sub_grid).astype(int)
                                basis = LLL.reduction(IntegerMatrix.from_matrix(sub_grid_int.tolist()))
                                rows, cols = basis.nrows, basis.ncols
                                list_of_lists = [[basis[i, j] for j in range(cols)] for i in range(rows)]
                                s1 = s_point(np.array(list_of_lists))
                                dist_s1 = dist_to_s(polyhedrons, s1)
                               
                                if dist_s1[0] >= 1:                                   
                                    print(dist_s1, s1, mat)
                                    list_dist.append(dist_s1)
                                    list_mat.append(mat.copy())
                                    list_lll.append(list_of_lists.copy())
                                    list_s1.append(s1.copy())

45
3 3 5
0
1
2
3
4
1 3 15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1 5 9
0
1
2
3
4
5
6
7
8
1 1 45
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [ ]:
# поиск приведенного базиса через функцию LLL

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


norm_length = 1 / max_len

for det_mat in range(46, 50):

    print(det_mat)

    list_all_factorizations = compute_factorizations(det_mat)
    list_diag_el = pad_lists_with_ones(list_all_factorizations)

    for diag_el in list_diag_el:
        mat = np.array([[diag_el[0], 0, 0, 0], 
                        [0, diag_el[1], 0, 0], 
                        [0, 0, diag_el[2], 0], 
                        [0, 0, 0, diag_el[3]]], float)
        max_num_col3 = diag_el[3]
        max_num_col2 = diag_el[2]
        max_num_col1 = diag_el[1]
        #max_num_col0 = diag_el[0]
        print(max_num_col1, max_num_col2, max_num_col3)

        for i1 in range(max_num_col3):
            mat[2][3] = i1
            print(i1)
            for i2 in range(max_num_col3):
                mat[1][3] = i2
                for i3 in range(max_num_col2):
                    mat[1][2] = i3
                    for i4 in range(max_num_col3):
                        mat[0][3] = i4
                        for i5 in range(max_num_col2):
                            mat[0][2] = i5
                            for i6 in range(max_num_col1):
                                mat[0][1] = i6
                                
                                sub_grid = np.dot(mat, grid)
                                basis = LLL(sub_grid)
                                
                                s1 = s_point(basis)
                                dist_s1 = dist_to_s(polyhedrons, s1)
                                
                                if dist_s1[0] >= 1:                                   
                                    print(dist_s1, s1, mat)
                                    list_dist.append(dist_s1)
                                    list_mat.append(mat.copy())
                                    list_lll.append(basis.copy())
                                    list_s1.append(s1.copy())

IndexError: list index out of range